# <center> IBM Applied Data Science Capstone Course by Coursera  </center>
###   <center> Opening a New Shopping Mall in Mumbai, India  </center>

  ###                                                                          <center>  By Arif Khan </center>

The Project invloves following steps :
* Build a dataframe of neighborhoods in Mumbai, India by web scraping the data from Wikipedia page.
* Get the geographical coordinates of the neighborhoods.
* Obtain the venue data for the neighborhoods from Foursquare API.
* Explore and cluster the neighborhoods.
* Select the best cluster to open a new shopping mall.

### 1. Import libraries

In [ ]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge geocoder --yes
!conda install -c conda-forge folium=0.5.0 --yes

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### 2. Scrap data from Wikipedia page into a DataFrame

In [7]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Mumbai").text

In [8]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [9]:
# create a list to store neighborhood data
neighborhoodList = []

In [10]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [11]:
# create a new DataFrame from the list
mumbai_df = pd.DataFrame({"Neighborhood": neighborhoodList})
mumbai_df.head(10)

,Neighborhood
0,List of neighbourhoods in Mumbai
1,Aarey Milk Colony
2,Agripada
3,Altamount Road
4,"Amboli, Mumbai"
5,Amrut Nagar
6,Antop Hill
7,Anushakti Nagar
8,Asalfa
9,Badhwar Park


In [12]:
# Droping row 0 and resetting index
mumbai_df.drop(mumbai_df.index[:1], inplace=True)
mumbai_df = mumbai_df.reset_index(drop=True)
mumbai_df.head(10)

,Neighborhood
0,Aarey Milk Colony
1,Agripada
2,Altamount Road
3,"Amboli, Mumbai"
4,Amrut Nagar
5,Antop Hill
6,Anushakti Nagar
7,Asalfa
8,Badhwar Park
9,Baiganwadi


In [13]:
# print the number of rows of the dataframe
mumbai_df.shape

(136, 1)

### 3. Get the geographical coordinates

In [14]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [15]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in mumbai_df["Neighborhood"].tolist() ]

In [16]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [17]:
# merge the coordinates into the original dataframe
mumbai_df['Latitude'] = df_coords['Latitude']
mumbai_df['Longitude'] = df_coords['Longitude']

In [31]:
# check the neighborhoods and the coordinates
print(mumbai_df.shape)
mumbai_df.head(10)

(136, 3)


,Neighborhood,Latitude,Longitude
0,Aarey Milk Colony,19.166115,72.859403
1,Agripada,18.976260,72.826150
2,Altamount Road,18.964338,72.807837
3,"Amboli, Mumbai",18.940170,72.834860
4,Amrut Nagar,19.145100,72.846750
5,Antop Hill,19.026350,72.866340
6,Anushakti Nagar,19.042830,72.927340
7,Asalfa,19.095240,72.892550
8,Badhwar Park,18.920250,72.826115
9,Baiganwadi,19.062930,72.926660


In [18]:
# save the DataFrame as CSV file
mumbai_df.to_csv("mumbai.csv", index=False)

### 4. Create a map of Mumbai with neighborhoods superimposed on top

In [19]:
# get the coordinates of Kuala Lumpur
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India: {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India: 18.9387711, 72.8353355.


In [20]:
# create map of Toronto using latitude and longitude values
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(mumbai_df['Latitude'], mumbai_df['Longitude'], mumbai_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mumbai)  
    
map_mumbai

In [21]:
# save the map as HTML file
map_mumbai.save('map_mumbai.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [53]:
# define Foursquare Credentials and Version
CLIENT_ID = 'CLIENT_ID: ' # your Foursquare ID
CLIENT_SECRET = 'CLIENT_SECRET:' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CLIENT_ID: 
CLIENT_SECRET:CLIENT_SECRET:


In [24]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(mumbai_df['Latitude'], mumbai_df['Longitude'], mumbai_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [25]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(11161, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Aarey Milk Colony,19.166115,72.859403,The Westin Mumbai Garden City,19.172654,72.860518,Hotel
1,Aarey Milk Colony,19.166115,72.859403,PVR Cinemas,19.174016,72.860485,Multiplex
2,Aarey Milk Colony,19.166115,72.859403,Starbucks,19.174177,72.860350,Coffee Shop
3,Aarey Milk Colony,19.166115,72.859403,British Brewing Company (BBC),19.174186,72.860504,Brewery
4,Aarey Milk Colony,19.166115,72.859403,MadOverDonuts,19.173902,72.860185,Donut Shop


##### Let's check how many venues were returned for each neighorhood

In [27]:
venues_df.groupby(["Neighborhood"]).count()
venues_df.head()

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Aarey Milk Colony,19.166115,72.859403,The Westin Mumbai Garden City,19.172654,72.860518,Hotel
1,Aarey Milk Colony,19.166115,72.859403,PVR Cinemas,19.174016,72.860485,Multiplex
2,Aarey Milk Colony,19.166115,72.859403,Starbucks,19.174177,72.860350,Coffee Shop
3,Aarey Milk Colony,19.166115,72.859403,British Brewing Company (BBC),19.174186,72.860504,Brewery
4,Aarey Milk Colony,19.166115,72.859403,MadOverDonuts,19.173902,72.860185,Donut Shop


##### Let's find out how many unique categories can be curated from all the returned venues

In [28]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 221 uniques categories.


In [29]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Hotel', 'Multiplex', 'Coffee Shop', 'Brewery', 'Donut Shop',
       'Shopping Mall', 'Breakfast Spot', 'Seafood Restaurant',
       'Hotel Bar', 'Fast Food Restaurant',
       'Vegetarian / Vegan Restaurant', 'Indian Restaurant', 'Bookstore',
       'Italian Restaurant', 'Bar', 'Snack Place', 'Bakery',
       'Sandwich Place', 'Farm', 'Chinese Restaurant', 'Asian Restaurant',
       'Clothing Store', 'Design Studio', 'Electronics Store', 'Gym',
       'Bistro', 'Gym / Fitness Center', 'Pizza Place', 'Café',
       'Restaurant', 'Ice Cream Shop', 'Smoke Shop',
       'American Restaurant', 'Dessert Shop', 'Diner', 'Train Station',
       'Nightclub', 'Club House', 'Golf Course',
       'Middle Eastern Restaurant', 'History Museum', 'Music Venue',
       'Scenic Lookout', 'Spa', 'Juice Bar', 'Deli / Bodega',
       'Bengali Restaurant', 'Cupcake Shop', 'Planetarium', 'Garden'],
      dtype=object)

In [30]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [31]:
# one hot encoding
mum_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mum_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mum_onehot.columns[-1]] + list(mum_onehot.columns[:-1])
mum_onehot = mum_onehot[fixed_columns]

print(mum_onehot.shape)
mum_onehot.head()

(11161, 222)


,Neighborhoods,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Water Park,Whisky Bar,Women's Store,Yoga Studio,Zoo
0,Aarey Milk Colony,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Aarey Milk Colony,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

##### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [32]:
mum_grouped = mum_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mum_grouped.shape)
mum_grouped

(136, 222)


,Neighborhoods,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Water Park,Whisky Bar,Women's Store,Yoga Studio,Zoo
0,Aarey Milk Colony,0.000000,0.00,0.00,0.013158,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,0.00,0.000000,0.000000,0.013158,0.000000,0.052632,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,0.026316,0.000000,0.00,0.000000,0.013158,0.013158,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.026316,0.00,0.000000,0.013158,0.013158,0.000000,0.000000,0.039474,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.013158,0.013158,0.000000,0.013158,0.026316,0.00,0.00,0.00,0.026316,0.000000,0.000000,0.013158,0.000000,0.118421,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [34]:
len(mum_grouped[mum_grouped["Shopping Mall"] > 0])

37

##### Create a new DataFrame for Shopping Mall data only

In [36]:
mum_mall = mum_grouped[["Neighborhoods","Shopping Mall"]]
mum_mall.head()

,Neighborhoods,Shopping Mall
0,Aarey Milk Colony,0.026316
1,Agripada,0.013514
2,Altamount Road,0.000000
3,"Amboli, Mumbai",0.000000
4,Amrut Nagar,0.000000


### 7. Cluster Neighborhoods

##### Run k-means to cluster the neighborhoods in Mumbai into 3 clusters.

In [37]:
# set number of clusters
kclusters = 3

mum_clustering = mum_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mum_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int32)

In [38]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
mum_merged = mum_mall.copy()

# add clustering labels
mum_merged["Cluster Labels"] = kmeans.labels_

In [39]:
mum_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mum_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Aarey Milk Colony,0.026316,1
1,Agripada,0.013514,1
2,Altamount Road,0.000000,0
3,"Amboli, Mumbai",0.000000,0
4,Amrut Nagar,0.000000,0


In [41]:
# merge mum_merged with mumbai_df to add latitude/longitude for each neighborhood
mum_merged = mum_merged.join(mumbai_df.set_index("Neighborhood"), on="Neighborhood")

print(mum_merged.shape)
mum_merged.head() # check the last columns!

(136, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Aarey Milk Colony,0.026316,1,19.166115,72.859403
1,Agripada,0.013514,1,18.976260,72.826150
2,Altamount Road,0.000000,0,18.964338,72.807837
3,"Amboli, Mumbai",0.000000,0,18.940170,72.834860
4,Amrut Nagar,0.000000,0,19.145100,72.846750


In [47]:
# sort the results by Cluster Labels
print(mum_merged.shape)
mum_merged.sort_values(["Cluster Labels"], inplace=True)
mum_merged.head(10)

(136, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
67,Kherwadi,0.0,0,19.059520,72.845510
102,Oshiwara,0.0,0,19.150540,72.830810
6,Anushakti Nagar,0.0,0,19.042830,72.927340
8,Badhwar Park,0.0,0,18.920250,72.826115
9,Baiganwadi,0.0,0,19.062930,72.926660
10,Ballard Estate,0.0,0,18.935410,72.839810
11,Bandra,0.0,0,19.054220,72.840190
12,Bandra Kurla Complex,0.0,0,19.062070,72.868490
14,Bhuleshwar,0.0,0,18.951300,72.829300
15,Bori Bunder,0.0,0,19.140257,72.804779


##### Finally, let's visualize the resulting clusters

In [48]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mum_merged['Latitude'], mum_merged['Longitude'], mum_merged['Neighborhood'], mum_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [49]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

In [50]:
# Cluster 0
mum_merged.loc[mum_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
67,Kherwadi,0.000000,0,19.059520,72.845510
102,Oshiwara,0.000000,0,19.150540,72.830810
6,Anushakti Nagar,0.000000,0,19.042830,72.927340
8,Badhwar Park,0.000000,0,18.920250,72.826115
9,Baiganwadi,0.000000,0,19.062930,72.926660
10,Ballard Estate,0.000000,0,18.935410,72.839810
11,Bandra,0.000000,0,19.054220,72.840190
12,Bandra Kurla Complex,0.000000,0,19.062070,72.868490
14,Bhuleshwar,0.000000,0,18.951300,72.829300
15,Bori Bunder,0.000000,0,19.140257,72.804779


In [51]:
# Cluster 1
mum_merged.loc[mum_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
105,Parel,0.030000,1,18.995660,72.839070
101,Orlem,0.016949,1,19.195740,72.835310
13,Bangur Nagar,0.030000,1,19.167840,72.832930
117,"Shastri Nagar, Goregaon",0.020000,1,19.160450,72.836310
0,Aarey Milk Colony,0.026316,1,19.166115,72.859403
28,Cotton Green,0.017241,1,18.985380,72.841210
23,Chinchpokli,0.030000,1,18.985940,72.831170
36,Dava Bazaar,0.022222,1,19.131400,72.926960
53,Guru Tegh Bahadur Nagar,0.022472,1,19.038360,72.864000
107,Poisar,0.020619,1,19.211530,72.852680


In [52]:
# Cluster 2
mum_merged.loc[mum_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
50,Gokuldham,0.047619,2,19.17227,72.869670
18,C.G.S. colony,0.045455,2,19.13891,72.938180
134,Yashodham,0.068493,2,19.19021,72.853661
84,Malad,0.053191,2,19.18655,72.848360
